In [1]:
import pandas as pd
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt

import gc
import warnings
import datetime as dt
import math
import seaborn as sn

import tensorflow as tf # import tensorflow as a whole
from tensorflow import keras # only import Keras

# import the dense and input layer
from keras.layers import Dense, Input
# the sequential model object
from keras.models import Sequential

from numpy import linalg as LA
import math

from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split

import tensorflow_addons as tfa

from keras.models import Sequential
from keras.layers import Input, CuDNNLSTM, Dense
from keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint



# load a saved model
from keras.models import load_model

In [2]:
#Import Data
data = pd.read_csv("data.csv")
samp_sol = pd.read_csv("sample_submission.csv")

In [3]:
data_set = pd.read_csv("data.csv")
display(data_set.head())

,id,author,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,0,Thompson,tube,7.00,3770.0,0.1754,NaN,10.8,432.0,3.6
1,1,Thompson,tube,NaN,6049.0,-0.0416,10.3,10.3,762.0,6.2
2,2,Thompson,NaN,13.79,2034.0,0.0335,7.7,7.7,457.0,2.5
3,3,Beus,annulus,13.79,3679.0,-0.0279,5.6,15.2,2134.0,3.0
4,4,NaN,tube,13.79,686.0,NaN,11.1,11.1,457.0,2.8


In [4]:
for k in data_set.columns:
    print("Column:", k)
    print(data_set[k].value_counts())
    print()

Column: id
0        1
21106    1
21104    1
21103    1
21102    1
        ..
10543    1
10542    1
10541    1
10540    1
31643    1
Name: id, Length: 31644, dtype: int64

Column: author
Thompson        17396
Janssen          2716
Weatherhead      2040
Beus             1604
Peskov           1084
Williams          891
Richenderfer      545
Mortimore         197
Kossolapov        101
Inasaka            46
Name: author, dtype: int64

Column: geometry
tube       21145
annulus     4381
plate        618
Name: geometry, dtype: int64

Column: pressure [MPa]
13.79    9226
6.89     4701
15.51    1117
10.34    1064
11.03     872
         ... 
3.77        1
12.21       1
7.06        1
7.70        1
12.14       1
Name: pressure [MPa], Length: 144, dtype: int64

Column: mass_flux [kg/m2-s]
4069.0    963
1519.0    634
1356.0    615
2034.0    533
1000.0    418
         ... 
835.0       1
967.0       1
6995.0      1
2050.0      1
1271.0      1
Name: mass_flux [kg/m2-s], Length: 733, dtype: int64

Column

In [5]:
max(np.array(data_set['x_e_out [-]']))

0.232

In [6]:
more_data = pd.read_csv("Data_CHF_Zhao_2020_ATE.csv")

In [7]:
for k in more_data.columns:
    print("Column:", k)
    print(more_data[k].value_counts())
    print()

Column: id
1       1
1240    1
1252    1
1251    1
1250    1
       ..
619     1
618     1
617     1
616     1
1865    1
Name: id, Length: 1865, dtype: int64

Column: author
Thompson        1202
Janssen          282
Weatherhead      162
Beus              77
Williams          51
Richenderfer      36
Mortimore         19
Peskov            17
Kossolapov        12
Inasaka            7
Name: author, dtype: int64

Column: geometry
tube       1439
annulus     378
plate        48
Name: geometry, dtype: int64

Column: pressure [MPa]
13.79    610
6.89     420
11.03     59
10.34     58
15.51     52
        ... 
10.57      1
10.93      1
12.34      1
12.38      1
10.24      1
Name: pressure [MPa], Length: 114, dtype: int64

Column: mass_flux [kg/m2-s]
4069    76
1519    55
2034    42
2292    28
1533    27
        ..
903      1
852      1
783      1
4164     1
7093     1
Name: mass_flux [kg/m2-s], Length: 578, dtype: int64

Column: x_e_out [-]
 0.0886    7
 0.0145    7
 0.0990    6
-0.0187    6
-0.

In [8]:
total_data = data_set.append(more_data).drop("id", axis=1)

/var/folders/tr/3zw9td7531z0_hhwc8105rv00000gn/T/ipykernel_11003/3739133671.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_data = data_set.append(more_data).drop("id", axis=1)


In [9]:
dummies_author = pd.get_dummies(total_data.author)
merged_with_authors = pd.concat([total_data, dummies_author], axis='columns')
merged_with_authors = merged_with_authors.drop(['author'], axis='columns')

dummies_geometry = pd.get_dummies(merged_with_authors.geometry)
merged_with_authors_and_geometry = pd.concat([merged_with_authors, dummies_geometry], axis='columns')
finalized_data = merged_with_authors_and_geometry.drop(['geometry'], axis='columns')

In [10]:
final_test = finalized_data[pd.isnull(finalized_data["x_e_out [-]"])]
final_test = final_test.fillna(0)
final_test = final_test.drop("x_e_out [-]", axis=1)
finalized_data = finalized_data.dropna(subset=["x_e_out [-]"])
finalized_data = finalized_data.fillna(0)

In [11]:
submission_indices = list(final_test.index)

## Train / Test Split

In [12]:
X = finalized_data.drop("x_e_out [-]", axis=1)
y = finalized_data["x_e_out [-]"]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [14]:
def convert_to_sequences(df):
    
    m,n = df.shape
    
    sequences = np.zeros((m, 2*n))
    
    i = 0
    for index, row in df.iterrows():
        
        k = 1
        j = 0
        for item in row: 
            if item != 0:
                sequences[i, j] = k
                sequences[i, j + 1] = item
                j += 2
            k += 1
        
        i+= 1

    return sequences 

In [15]:
X_train = convert_to_sequences(X_train)
X_test = convert_to_sequences(X_test)

In [16]:
X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)

y_train = np.array(y_train)
y_test = np.array(y_test)

In [17]:
X_train.shape

(20784, 1, 38)

## Construct Model

In [18]:
model = Sequential()
model.add(LSTM(64, input_shape=(1,2*19)))
model.add(Dense(1, activation='tanh'))

2023-05-29 10:45:38.417990: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
model.compile(loss=['mean_squared_error'] , optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                26368     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 26,433
Trainable params: 26,433
Non-trainable params: 0
_________________________________________________________________


In [20]:
#model = load_model('best_model.h5')

In [37]:
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

In [46]:
model.fit(X_train, y_train, epochs=8000, batch_size=20, verbose=2, validation_split=0.1, callbacks=[mc])

Epoch 1/8000

Epoch 1: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0077 - accuracy: 3.2077e-04 - val_loss: 0.0082 - val_accuracy: 0.0000e+00 - 1s/epoch - 2ms/step
Epoch 2/8000

Epoch 2: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0078 - accuracy: 3.2077e-04 - val_loss: 0.0084 - val_accuracy: 0.0000e+00 - 770ms/epoch - 823us/step
Epoch 3/8000

Epoch 3: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0078 - accuracy: 3.2077e-04 - val_loss: 0.0085 - val_accuracy: 0.0000e+00 - 774ms/epoch - 827us/step
Epoch 4/8000

Epoch 4: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0078 - accuracy: 3.2077e-04 - val_loss: 0.0084 - val_accuracy: 0.0000e+00 - 776ms/epoch - 829us/step
Epoch 5/8000

Epoch 5: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0077 - accuracy: 3.2077e-04 - val_loss: 0.0083 - val_accuracy: 0.0000e+00 - 779ms/epoch - 832us/step
Epoch 6/8000

Epoch 6: val_loss did not improve from 0.00788
936/936 - 1s - loss

Epoch 45/8000

Epoch 45: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0077 - accuracy: 3.2077e-04 - val_loss: 0.0082 - val_accuracy: 0.0000e+00 - 772ms/epoch - 824us/step
Epoch 46/8000

Epoch 46: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0078 - accuracy: 3.2077e-04 - val_loss: 0.0085 - val_accuracy: 0.0000e+00 - 770ms/epoch - 822us/step
Epoch 47/8000

Epoch 47: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0079 - accuracy: 3.2077e-04 - val_loss: 0.0082 - val_accuracy: 0.0000e+00 - 768ms/epoch - 820us/step
Epoch 48/8000

Epoch 48: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0078 - accuracy: 3.2077e-04 - val_loss: 0.0083 - val_accuracy: 0.0000e+00 - 772ms/epoch - 825us/step
Epoch 49/8000

Epoch 49: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0080 - accuracy: 3.2077e-04 - val_loss: 0.0081 - val_accuracy: 0.0000e+00 - 771ms/epoch - 824us/step
Epoch 50/8000

Epoch 50: val_loss did not improve from 0.00788
93

Epoch 89/8000

Epoch 89: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0078 - accuracy: 3.2077e-04 - val_loss: 0.0083 - val_accuracy: 0.0000e+00 - 770ms/epoch - 822us/step
Epoch 90/8000

Epoch 90: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0078 - accuracy: 3.2077e-04 - val_loss: 0.0082 - val_accuracy: 0.0000e+00 - 768ms/epoch - 820us/step
Epoch 91/8000

Epoch 91: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0077 - accuracy: 3.2077e-04 - val_loss: 0.0081 - val_accuracy: 0.0000e+00 - 775ms/epoch - 828us/step
Epoch 92/8000

Epoch 92: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0077 - accuracy: 3.2077e-04 - val_loss: 0.0081 - val_accuracy: 0.0000e+00 - 769ms/epoch - 822us/step
Epoch 93/8000

Epoch 93: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0077 - accuracy: 3.2077e-04 - val_loss: 0.0082 - val_accuracy: 0.0000e+00 - 769ms/epoch - 821us/step
Epoch 94/8000

Epoch 94: val_loss did not improve from 0.00788
93

Epoch 133/8000

Epoch 133: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0079 - accuracy: 3.2077e-04 - val_loss: 0.0084 - val_accuracy: 0.0000e+00 - 772ms/epoch - 824us/step
Epoch 134/8000

Epoch 134: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0079 - accuracy: 3.2077e-04 - val_loss: 0.0083 - val_accuracy: 0.0000e+00 - 775ms/epoch - 828us/step
Epoch 135/8000

Epoch 135: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0078 - accuracy: 3.2077e-04 - val_loss: 0.0081 - val_accuracy: 0.0000e+00 - 769ms/epoch - 822us/step
Epoch 136/8000

Epoch 136: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0078 - accuracy: 3.2077e-04 - val_loss: 0.0085 - val_accuracy: 0.0000e+00 - 768ms/epoch - 821us/step
Epoch 137/8000

Epoch 137: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0080 - accuracy: 3.2077e-04 - val_loss: 0.0083 - val_accuracy: 0.0000e+00 - 777ms/epoch - 830us/step
Epoch 138/8000

Epoch 138: val_loss did not improve fro

Epoch 177/8000

Epoch 177: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0078 - accuracy: 3.2077e-04 - val_loss: 0.0083 - val_accuracy: 0.0000e+00 - 775ms/epoch - 828us/step
Epoch 178/8000

Epoch 178: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0078 - accuracy: 3.2077e-04 - val_loss: 0.0084 - val_accuracy: 0.0000e+00 - 768ms/epoch - 821us/step
Epoch 179/8000

Epoch 179: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0078 - accuracy: 3.2077e-04 - val_loss: 0.0083 - val_accuracy: 0.0000e+00 - 768ms/epoch - 821us/step
Epoch 180/8000

Epoch 180: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0079 - accuracy: 3.2077e-04 - val_loss: 0.0085 - val_accuracy: 0.0000e+00 - 780ms/epoch - 833us/step
Epoch 181/8000

Epoch 181: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0079 - accuracy: 3.2077e-04 - val_loss: 0.0083 - val_accuracy: 0.0000e+00 - 771ms/epoch - 823us/step
Epoch 182/8000

Epoch 182: val_loss did not improve fro

...

936/936 - 1s - loss: 0.0076 - accuracy: 3.2077e-04 - val_loss: 0.0084 - val_accuracy: 0.0000e+00 - 785ms/epoch - 838us/step
Epoch 5227/8000

Epoch 5227: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0076 - accuracy: 3.2077e-04 - val_loss: 0.0084 - val_accuracy: 0.0000e+00 - 783ms/epoch - 836us/step
Epoch 5228/8000

Epoch 5228: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0076 - accuracy: 3.2077e-04 - val_loss: 0.0085 - val_accuracy: 0.0000e+00 - 774ms/epoch - 827us/step
Epoch 5229/8000

Epoch 5229: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0075 - accuracy: 3.2077e-04 - val_loss: 0.0084 - val_accuracy: 0.0000e+00 - 789ms/epoch - 843us/step
Epoch 5230/8000

Epoch 5230: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0077 - accuracy: 3.2077e-04 - val_loss: 0.0083 - val_accuracy: 0.0000e+00 - 778ms/epoch - 832us/step
Epoch 5231/8000

Epoch 5231: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0076 - accuracy: 3.207

936/936 - 1s - loss: 0.0076 - accuracy: 3.2077e-04 - val_loss: 0.0084 - val_accuracy: 0.0000e+00 - 768ms/epoch - 820us/step
Epoch 5270/8000

Epoch 5270: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0075 - accuracy: 3.2077e-04 - val_loss: 0.0085 - val_accuracy: 0.0000e+00 - 764ms/epoch - 816us/step
Epoch 5271/8000

Epoch 5271: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0076 - accuracy: 3.2077e-04 - val_loss: 0.0083 - val_accuracy: 0.0000e+00 - 772ms/epoch - 824us/step
Epoch 5272/8000

Epoch 5272: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0077 - accuracy: 3.2077e-04 - val_loss: 0.0082 - val_accuracy: 0.0000e+00 - 767ms/epoch - 819us/step
Epoch 5273/8000

Epoch 5273: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0076 - accuracy: 3.2077e-04 - val_loss: 0.0084 - val_accuracy: 0.0000e+00 - 774ms/epoch - 827us/step
Epoch 5274/8000

Epoch 5274: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0076 - accuracy: 3.207

936/936 - 1s - loss: 0.0076 - accuracy: 3.2077e-04 - val_loss: 0.0082 - val_accuracy: 0.0000e+00 - 769ms/epoch - 821us/step
Epoch 5313/8000

Epoch 5313: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0077 - accuracy: 3.2077e-04 - val_loss: 0.0085 - val_accuracy: 0.0000e+00 - 767ms/epoch - 820us/step
Epoch 5314/8000

Epoch 5314: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0076 - accuracy: 3.2077e-04 - val_loss: 0.0086 - val_accuracy: 0.0000e+00 - 764ms/epoch - 816us/step
Epoch 5315/8000

Epoch 5315: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0077 - accuracy: 3.2077e-04 - val_loss: 0.0085 - val_accuracy: 0.0000e+00 - 777ms/epoch - 831us/step
Epoch 5316/8000

Epoch 5316: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0075 - accuracy: 3.2077e-04 - val_loss: 0.0085 - val_accuracy: 0.0000e+00 - 761ms/epoch - 813us/step
Epoch 5317/8000

Epoch 5317: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0076 - accuracy: 3.207

936/936 - 1s - loss: 0.0075 - accuracy: 3.2077e-04 - val_loss: 0.0083 - val_accuracy: 0.0000e+00 - 824ms/epoch - 880us/step
Epoch 5356/8000

Epoch 5356: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0076 - accuracy: 3.2077e-04 - val_loss: 0.0084 - val_accuracy: 0.0000e+00 - 751ms/epoch - 803us/step
Epoch 5357/8000

Epoch 5357: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0078 - accuracy: 3.2077e-04 - val_loss: 0.0084 - val_accuracy: 0.0000e+00 - 733ms/epoch - 783us/step
Epoch 5358/8000

Epoch 5358: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0077 - accuracy: 3.2077e-04 - val_loss: 0.0084 - val_accuracy: 0.0000e+00 - 748ms/epoch - 799us/step
Epoch 5359/8000

Epoch 5359: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0076 - accuracy: 3.2077e-04 - val_loss: 0.0089 - val_accuracy: 0.0000e+00 - 726ms/epoch - 775us/step
Epoch 5360/8000

Epoch 5360: val_loss did not improve from 0.00788
936/936 - 1s - loss: 0.0077 - accuracy: 3.207

KeyboardInterrupt: 

In [47]:
model = load_model('best_model.h5')

In [48]:
y_hat = model.predict(X_test)

73/73 [==============================] - 0s 465us/step


In [49]:
math.sqrt(np.mean(np.square(y_test - y_hat)))# / math.sqrt(np.mean(np.square(y_test)))

0.1178507176775874

# Perform Prediction

In [50]:
X_submission = convert_to_sequences(final_test)
X_submission = np.expand_dims(X_submission, axis=1)

In [51]:
y_submission = model.predict(X_submission)

326/326 [==============================] - 0s 403us/step


In [52]:
y_submission = list(y_submission.reshape(len(y_submission)))

In [53]:
df = pd.DataFrame(data={"id": submission_indices, "x_e_out [-]": y_submission})
df.to_csv("submission.csv", sep=',',index=False)